In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Load dataframe from .pickle file
play_file_pickle = '/Users/ZhijingYe/Desktop/data/output/reduced_play_log.pkl'
df_play = pd.read_pickle(play_file_pickle)

In [3]:
df_play.head()

,uid,device,song_type,play_time,song_length,label,date,song_id_new,singer_popularity_rank
0,168308107,ar,0,296,296,0,20170410,816,183
1,168112765,ar,0,272,0,0,20170410,13121,35
2,168274411,ar,0,24,156,0,20170410,140473,25366
3,168274411,ar,0,333,334,0,20170410,1171,431
4,168274411,ar,0,155,156,0,20170410,140473,25366


In [4]:
# For churn prediction, choose the logs before the cut-off date 20170422
df_play = df_play.loc[df_play.date < '20170422']

In [5]:
df_play.date.unique()

array(['20170410', '20170411', '20170401', '20170414', '20170418',
       '20170408', '20170404', '20170308', '20170415', '20170419',
       '20170409', '20170305', '20170405', '20170302', '20170402',
       '20170412', '20170403', '20170413', '20170331', '20170406',
       '20170416', '20170421', '20170307', '20170407', '20170417',
       '20170420', '20170309', '20170301', '20170304', '20170330',
       '20170306', '20170303', '20170339'], dtype=object)

In [13]:
df_play_1.date = df_play.date.copy()

In [14]:
df_play_1.date.sort()

In [17]:
print(df_play_1.date.unique())

['20170301' '20170302' '20170303' '20170304' '20170305' '20170306'
 '20170307' '20170308' '20170309' '20170330' '20170331' '20170339'
 '20170401' '20170402' '20170403' '20170404' '20170405' '20170406'
 '20170407' '20170408' '20170409' '20170410' '20170411' '20170412'
 '20170413' '20170414' '20170415' '20170416' '20170417' '20170418'
 '20170419' '20170420' '20170421']


### Two features to create: 1. rare_song_player (taking 0,1), and 2. song_popular_ratio

#### Get the set of most popular songs

In [18]:
number_of_times_played_dist = df_play[['uid','song_id_new']].groupby('song_id_new').count()\
                            .rename(columns={"uid": "play_count"})
                              

In [19]:
number_of_times_played_dist.reset_index(inplace = True)

In [20]:
number_of_times_played_dist.head()

,song_id_new,play_count
0,1,143771
1,2,113161
2,3,75450
3,4,61404
4,5,55509


In [21]:
number_of_times_played_dist.tail()

,song_id_new,play_count
764739,911024,1
764740,911025,1
764741,911026,1
764742,911027,1
764743,911028,1


In [22]:
bins = [0, 1, 100, 1000, 10000, 100000]

In [23]:
group_names = ['1','1~100', '100~1000', '1000~10000', '10000~10000']

In [24]:
number_of_times_played_dist['group'] = pd.cut(number_of_times_played_dist.play_count, bins, labels=group_names)

In [25]:
number_of_times_played_dist.group.value_counts()

1~100          475896
1              266344
100~1000        20207
1000~10000       2216
10000~10000        79
dtype: int64

In [26]:
# Calculate the percentage of each group
number_of_times_played_dist.group.value_counts()/number_of_times_played_dist.shape[0]

1~100          0.622295
1              0.348279
100~1000       0.026423
1000~10000     0.002898
10000~10000    0.000103
dtype: float64

#### Most songs are played less than 100 times, thus we could set songs played more than 1000 times as popular songs, top 0.25%.

In [27]:
# Popular songs:
songs_id_popular = set(number_of_times_played_dist\
                    [number_of_times_played_dist['group'].isin(['1000~10000','10000~10000'])]\
                    .song_id_new)

In [28]:
len(songs_id_popular)

2295

In [29]:
df_play['song_popular'] = df_play.song_id_new.isin(songs_id_popular)
df_play.head()

,uid,device,song_type,play_time,song_length,label,date,song_id_new,singer_popularity_rank,song_popular
0,168308107,ar,0,296,296,0,20170410,816,183,True
1,168112765,ar,0,272,0,0,20170410,13121,35,False
2,168274411,ar,0,24,156,0,20170410,140473,25366,False
3,168274411,ar,0,333,334,0,20170410,1171,431,True
4,168274411,ar,0,155,156,0,20170410,140473,25366,False


In [30]:
# Make a new dataframe with each uid summarized behavior as one column
df = pd.DataFrame(columns= ['uid'])

In [31]:
df.uid = df_play.groupby(['uid']).size().index

In [32]:
df.head()

,uid
0,100052111
1,100107633
2,100223381
3,100581940
4,100733657


In [33]:
df_song_popular = df_play[['uid','song_popular']].groupby('uid').mean().reset_index()

In [34]:
df_song_popular.head()

,uid,song_popular
0,100052111,0.107143
1,100107633,0.500000
2,100223381,0.000000
3,100581940,0.800000
4,100733657,0.434783


In [35]:
df = df.merge(df_song_popular, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular
0,100052111,0.107143
1,100107633,0.500000
2,100223381,0.000000
3,100581940,0.800000
4,100733657,0.434783


In [36]:
#majority vote of device
df_device = df_play[['uid', 'device']].groupby('uid')['device'].apply(lambda x: x.value_counts().index[0])
df_device.head()

uid
100052111     ar
100107633     ar
100223381     ar
100581940     ar
100733657     ar
Name: device, dtype: object

In [37]:
df_device = df_device.reset_index()

In [38]:
df_device.head()

,uid,device
0,100052111,ar
1,100107633,ar
2,100223381,ar
3,100581940,ar
4,100733657,ar


In [39]:
df_device.device.value_counts()

ar    131106
ip     19743
dtype: int64

In [40]:
# majority vote of song_type
df_song_type = df_play[['uid', 'song_type']].groupby('uid')['song_type']\
                .apply(lambda x: x.value_counts().index[0]).reset_index()
df_song_type.head()

,uid,song_type
0,100052111,0
1,100107633,0
2,100223381,0
3,100581940,0
4,100733657,0


In [41]:
df_song_type.song_type.value_counts()

0    138272
1     11047
2      1492
3        38
dtype: int64

In [42]:
# Merge device into df
df = df.merge(df_device, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular,device
0,100052111,0.107143,ar
1,100107633,0.500000,ar
2,100223381,0.000000,ar
3,100581940,0.800000,ar
4,100733657,0.434783,ar


In [43]:
# Merge song_type into df
df = df.merge(df_song_type, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular,device,song_type
0,100052111,0.107143,ar,0
1,100107633,0.500000,ar,0
2,100223381,0.000000,ar,0
3,100581940,0.800000,ar,0
4,100733657,0.434783,ar,0


In [44]:
df = df.rename(columns={'song_type' : 'major_song_type'})

In [45]:
df_song_popularity = df_play[['uid','song_popular']].groupby('uid')\
                    .mean().rename(columns = {'song_popular' : 'song_popularity_ratio'})


In [46]:
df_song_popularity = df_song_popularity.reset_index()
df_song_popularity.head()

,uid,song_popularity_ratio
0,100052111,0.107143
1,100107633,0.500000
2,100223381,0.000000
3,100581940,0.800000
4,100733657,0.434783


In [47]:
# Merge song_popularity_ratio into df
df = df.merge(df_song_popularity, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio
0,100052111,0.107143,ar,0,0.107143
1,100107633,0.500000,ar,0,0.500000
2,100223381,0.000000,ar,0,0.000000
3,100581940,0.800000,ar,0,0.800000
4,100733657,0.434783,ar,0,0.434783


In [48]:
# Total Play_time
df_total_play_time = df_play[['uid','play_time']].groupby('uid').sum()/60# change time unit to minutes


In [49]:
df_total_play_time = df_total_play_time.rename(columns = {'play_time':'total_play_time'}).reset_index()

In [50]:
df_total_play_time.head()

,uid,total_play_time
0,100052111,0.000000
1,100107633,3.033333
2,100223381,33.333333
3,100581940,17.200000
4,100733657,374.766667


In [51]:
sum(df_total_play_time.total_play_time>100000)

1254

In [52]:
df = df.merge(df_total_play_time, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time
0,100052111,0.107143,ar,0,0.107143,0.000000
1,100107633,0.500000,ar,0,0.500000,3.033333
2,100223381,0.000000,ar,0,0.000000,33.333333
3,100581940,0.800000,ar,0,0.800000,17.200000
4,100733657,0.434783,ar,0,0.434783,374.766667


In [53]:

df_play[['label']] = df_play[['label']].astype('int')
df_label = df_play[['uid','label']].groupby('uid').mean().reset_index()
df_label.head()

,uid,label
0,100052111,1
1,100107633,1
2,100223381,1
3,100581940,1
4,100733657,1


In [54]:
# Merge label into df
df = df.merge(df_label, left_on = 'uid', right_on= 'uid', how = 'inner')
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label
0,100052111,0.107143,ar,0,0.107143,0.000000,1
1,100107633,0.500000,ar,0,0.500000,3.033333,1
2,100223381,0.000000,ar,0,0.000000,33.333333,1
3,100581940,0.800000,ar,0,0.800000,17.200000,1
4,100733657,0.434783,ar,0,0.434783,374.766667,1


In [55]:
# Total play_time
# Create bins for total_play_time
bins = [-1,100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,float('inf')]
bin_names = ['0-100','100-200','200-300','300-400','400-500','500-600','600-700','700-800','800-900','900-1k',\
             '1k-2k','2k-3k','3k-4k','4k-5k','5k-6k','6k-7k','7k-8k','8k-9k','9k-10k','10k+']
df['play_time_label'] = pd.cut(df.total_play_time,bins, labels = bin_names)
df.play_time_label.value_counts()

0-100      89283
100-200    17637
200-300     8952
1k-2k       6196
300-400     6006
400-500     4401
500-600     3230
10k+        3112
600-700     2715
700-800     2160
2k-3k       1910
800-900     1728
900-1k      1393
3k-4k        832
4k-5k        444
5k-6k        276
6k-7k        176
7k-8k        142
8k-9k        112
9k-10k        89
dtype: int64

In [56]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label
0,100052111,0.107143,ar,0,0.107143,0.000000,1,0-100
1,100107633,0.500000,ar,0,0.500000,3.033333,1,0-100
2,100223381,0.000000,ar,0,0.000000,33.333333,1,0-100
3,100581940,0.800000,ar,0,0.800000,17.200000,1,0-100
4,100733657,0.434783,ar,0,0.434783,374.766667,1,300-400


In [57]:
df[['play_time_label', 'label']].groupby('play_time_label').mean()

,label
play_time_label,
0-100,0.824155
100-200,0.635141
200-300,0.548481
300-400,0.505994
400-500,0.476028
500-600,0.424768
600-700,0.368324
700-800,0.368519
800-900,0.354167


#### There is definately dependence between churn rate and total_play_time. <br> And the churn rate is similar for total_play_time = (100, 400], (400, 700], (700, 1k], (1k, 7k], (7k, 10k], (10k+] 

In [58]:
df = df.drop(['play_time_label'], axis = 1)

In [59]:
# Reduce number of total_play_time bins. 
bins = [-1,100,400,700,1000,7000,10000,float('inf')]
bin_names = ['0-100','100-400','400-700','700-1k','1k-7k','7k-10k','10k+']
df['play_time_label'] = pd.cut(df.total_play_time,bins, labels = bin_names)
df.play_time_label.value_counts()

0-100      89283
100-400    32595
400-700    10346
1k-7k       9834
700-1k      5281
10k+        3112
7k-10k       343
dtype: int64

In [60]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label
0,100052111,0.107143,ar,0,0.107143,0.000000,1,0-100
1,100107633,0.500000,ar,0,0.500000,3.033333,1,0-100
2,100223381,0.000000,ar,0,0.000000,33.333333,1,0-100
3,100581940,0.800000,ar,0,0.800000,17.200000,1,0-100
4,100733657,0.434783,ar,0,0.434783,374.766667,1,100-400


In [61]:
df[['play_time_label', 'label']].groupby('play_time_label').mean()

,label
play_time_label,
0-100,0.824155
100-400,0.587543
400-700,0.431761
700-1k,0.343306
1k-7k,0.273134
7k-10k,0.469388
10k+,0.813946


In [62]:
# Average play_time
df_avg_play_time = df_play[['uid', 'play_time']].\
                    groupby('uid').mean().rename(columns = {'play_time' : 'avg_play_time'}).reset_index()

In [63]:
df_avg_play_time.head()

,uid,avg_play_time
0,100052111,0.000000
1,100107633,45.500000
2,100223381,125.000000
3,100581940,51.600000
4,100733657,195.530435


In [64]:
# Merge avg_play_time into df
df = df.merge(df_avg_play_time, left_on = 'uid', right_on = 'uid', how = 'inner')

In [65]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time
0,100052111,0.107143,ar,0,0.107143,0.000000,1,0-100,0.000000
1,100107633,0.500000,ar,0,0.500000,3.033333,1,0-100,45.500000
2,100223381,0.000000,ar,0,0.000000,33.333333,1,0-100,125.000000
3,100581940,0.800000,ar,0,0.800000,17.200000,1,0-100,51.600000
4,100733657,0.434783,ar,0,0.434783,374.766667,1,100-400,195.530435


In [66]:
# Create a set for the rarely played songs, which are played once in the entire dataframe
num_song_least_pop = sum(df_play.groupby('song_id_new').size()<=1)
num_song_least_pop

266344

In [67]:
least_popular_songs = df_play['song_id_new'].value_counts()[-1:-num_song_least_pop-1:-1].index

In [68]:
# Create a new column in original df_play to indicate whether the played song is in the least popular set.
df_play['least_popular'] = df_play.song_id_new.isin(set(least_popular_songs))

In [69]:
# For each uid, count the number of times played the least popular song
df_least_pop_counts = df_play[['uid', 'least_popular']].groupby('uid').\
                        sum().rename(columns = {'least_popular' : 'least_pop_count'}).reset_index()

In [70]:
df_least_pop_counts.head()

,uid,least_pop_count
0,100052111,0
1,100107633,0
2,100223381,6
3,100581940,1
4,100733657,0


In [71]:
# Merge the least_pop_count into df
df = df.merge(df_least_pop_counts, left_on = 'uid', right_on = 'uid', how = 'inner')

In [72]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count
0,100052111,0.107143,ar,0,0.107143,0.000000,1,0-100,0.000000,0
1,100107633,0.500000,ar,0,0.500000,3.033333,1,0-100,45.500000,0
2,100223381,0.000000,ar,0,0.000000,33.333333,1,0-100,125.000000,6
3,100581940,0.800000,ar,0,0.800000,17.200000,1,0-100,51.600000,1
4,100733657,0.434783,ar,0,0.434783,374.766667,1,100-400,195.530435,0


In [73]:
df.least_pop_count.value_counts()

0     106520
1      15651
2       7710
3       4699
4       3063
5       2206
6       1579
7       1266
8       1011
9        842
10       684
11       602
12       535
13       387
14       346
...
143    1
150    1
151    1
157    1
159    1
161    1
162    1
163    1
164    1
166    1
168    1
171    1
172    1
178    1
824    1
Length: 198, dtype: int64

In [74]:
# Add a column in df to show the total play count
df['total_play_count'] = list(df_play.groupby('uid').size())

In [75]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count,total_play_count
0,100052111,0.107143,ar,0,0.107143,0.000000,1,0-100,0.000000,0,112
1,100107633,0.500000,ar,0,0.500000,3.033333,1,0-100,45.500000,0,4
2,100223381,0.000000,ar,0,0.000000,33.333333,1,0-100,125.000000,6,16
3,100581940,0.800000,ar,0,0.800000,17.200000,1,0-100,51.600000,1,20
4,100733657,0.434783,ar,0,0.434783,374.766667,1,100-400,195.530435,0,115


In [76]:
df['least_popular_ratio'] = df.apply(lambda x: x.least_pop_count/x.total_play_count, axis = 1)
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count,total_play_count,least_popular_ratio
0,100052111,0.107143,ar,0,0.107143,0.000000,1,0-100,0.000000,0,112,0.000
1,100107633,0.500000,ar,0,0.500000,3.033333,1,0-100,45.500000,0,4,0.000
2,100223381,0.000000,ar,0,0.000000,33.333333,1,0-100,125.000000,6,16,0.375
3,100581940,0.800000,ar,0,0.800000,17.200000,1,0-100,51.600000,1,20,0.050
4,100733657,0.434783,ar,0,0.434783,374.766667,1,100-400,195.530435,0,115,0.000


In [91]:
## save the least popular ratio file to pickle

import pickle

f = open('/Users/ZhijingYe/Desktop/data/output/least_popular_ratio.pckl', 'wb')
pickle.dump(df.least_pop_count/df.total_play_count, f)
f.close()

In [92]:
f = open('/Users/ZhijingYe/Desktop/data/output/least_popular_ratio.pckl', 'rb')
obj = pickle.load(f)
f.close()

In [93]:
df.least_popular_ratio_1 = obj.copy()

In [94]:
set(df.least_popular_ratio_1).intersection(df.least_popular_ratio)

{0.0,
 0.375,
 0.0056603773584905656,
 0.5,
 1.0,
 0.02004008016032064,
 0.42692307692307691,
 0.017301038062283738,
 0.0020449897750511249,
 0.625,
 0.14772727272727273,
 0.056603773584905662,
 0.1888111888111888,
 0.0038002171552660152,
 0.31192660550458717,
 0.00096061479346781938,
 0.052287581699346407,
 0.041580041580041582,
 0.0050100200400801601,
 0.12477064220183487,
 0.047888386123680245,
 0.0466786355475763,
 0.093126385809312637,
 0.0022354694485842027,
 0.031446540880503145,
 0.018252933507170794,
 0.013698630136986301,
 0.029919447640966629,
 0.0072022160664819944,
 0.0174496644295302,
 0.00911854103343465,
 0.14583333333333334,
 0.0029806259314456036,
 0.087209302325581398,
 0.00031625553447185326,
 0.0033745781777277839,
 0.044144144144144144,
 0.13238289205702647,
 0.036939313984168866,
 0.01511879049676026,
 0.098591549295774641,
 0.00054318305268875606,
 0.079051383399209488,
 0.21495327102803738,
 0.00057636887608069167,
 0.0052137643378519288,
 0.068062827225130892,

In [98]:
len(df.least_popular_ratio == df.least_popular_ratio_1)

150849

In [95]:
print(df.least_popular_ratio_1)

0     0.000000
1     0.000000
2     0.375000
3     0.050000
4     0.000000
5     0.400000
6     0.000000
7     0.210526
8     0.085366
9     0.000000
10    0.000000
11    0.000000
12    0.014706
13    0.000000
14    0.000000
...
150834    0.000000
150835    0.005354
150836    0.300000
150837    0.000000
150838    0.035714
150839    0.009901
150840    0.458333
150841    0.016807
150842    0.591837
150843    0.000000
150844    0.003003
150845    0.130435
150846    0.000000
150847    0.006289
150848    0.000000
Length: 150849, dtype: float64


In [99]:
print(df.least_popular_ratio)

0     0.000000
1     0.000000
2     0.375000
3     0.050000
4     0.000000
5     0.400000
6     0.000000
7     0.210526
8     0.085366
9     0.000000
10    0.000000
11    0.000000
12    0.014706
13    0.000000
14    0.000000
...
150834    0.000000
150835    0.005354
150836    0.300000
150837    0.000000
150838    0.035714
150839    0.009901
150840    0.458333
150841    0.016807
150842    0.591837
150843    0.000000
150844    0.003003
150845    0.130435
150846    0.000000
150847    0.006289
150848    0.000000
Name: least_popular_ratio, Length: 150849, dtype: float64


In [121]:
# Examine the dependence of churn rate on playing least popular songs:
df.groupby(pd.cut(df.least_popular_ratio,\
          np.percentile(df.least_popular_ratio,[0,71,80,90,100]), include_lowest=True, ))\
          .mean()['label']

# The dependence of churn-rate on least_popular_ratio is not clear, yet...
# for [0,71,80,90,100],the numbers are tested here, if not applicable, then there will be two the same figure for two variation

least_popular_ratio
[0, 0.00151]           0.744190
(0.00151, 0.018]       0.373106
(0.018, 0.069]         0.564356
(0.069, 1]             0.740572
Name: label, dtype: float64

In [111]:
# Examine the dependence of churn rate on playing least popular songs:
df['range'] = pd.cut(df.least_popular_ratio,\
          df.least_popular_ratio.quantile([0,0.8,0.9,1]), include_lowest=True )
print(df)
# The dependence of churn-rate on least_popular_ratio is not clear, yet...

               uid  song_popular device major_song_type  \
0        100052111      0.107143     ar               0   
1        100107633      0.500000     ar               0   
2        100223381      0.000000     ar               0   
3        100581940      0.800000     ar               0   
4       100733657       0.434783     ar               0   
5       100920280       0.000000     ar               1   
6        101122382      0.000000     ar               0   
7        101169218      0.263158     ar               0   
8        101372876      0.003049     ar               1   
9       101374971       0.025641     ar               2   
10      101537348       0.533333     ar               0   
11       101703605      0.545455     ar               0   
12       101907949      0.161765     ar               0   
13      101986413       0.461538     ar               0   
14       102014109      0.066531     ar               1   
15       10214468       0.340909     ar               0 

#### Examine the cutoff number of play logs for labeling popular songs: 100, 500,1000,2000

In [126]:
popular_songs_cutoff = [100,500,1000,2000]
cutoff_quantile_dict = {}
for i in range(len(popular_songs_cutoff)):
    print 'procesing no.%d cutoff: %d' % (i,popular_songs_cutoff[i])
    num_pop_songs = sum(df_play.groupby('song_id_new').size()>popular_songs_cutoff[i])
    popular_songs = df_play['song_id_new'].value_counts()[:(num_pop_songs-1)].index
    df_play['is_popular'] = df_play.song_id_new.isin(set(popular_songs))
    df_temp = df_play.groupby('uid').mean()
    cutoff_quantile_dict[i] = df_temp.groupby(pd.cut(df_temp.is_popular, \
                           np.percentile(df_temp.is_popular, [0,25,50,73,100]),include_lowest = True)).mean()['label']
cutoff_quantile_dict

procesing no.0 cutoff: 100
procesing no.1 cutoff: 500
procesing no.2 cutoff: 1000
procesing no.3 cutoff: 2000


{0: is_popular
 [0, 0.521]        0.699785
 (0.521, 0.803]    0.598016
 (0.803, 0.993]    0.584914
 (0.993, 1]        0.864519
 Name: label, dtype: float64, 1: is_popular
 [0, 0.25]         0.719074
 (0.25, 0.545]     0.606984
 (0.545, 0.802]    0.621590
 (0.802, 1]        0.806090
 Name: label, dtype: float64, 2: is_popular
 [0, 0.149]        0.731088
 (0.149, 0.412]    0.608505
 (0.412, 0.673]    0.624869
 (0.673, 1]        0.791853
 Name: label, dtype: float64, 3: is_popular
 [0, 0.068]        0.748416
 (0.068, 0.277]    0.597699
 (0.277, 0.521]    0.628730
 (0.521, 1]        0.782464
 Name: label, dtype: float64}

In [127]:
pd.DataFrame({popular_songs_cutoff[0]:list(cutoff_quantile_dict[0]),
             popular_songs_cutoff[1]:list(cutoff_quantile_dict[1]),
             popular_songs_cutoff[2]:list(cutoff_quantile_dict[2]),
             popular_songs_cutoff[3]:list(cutoff_quantile_dict[3])})

,100,500,1000,2000
0,0.699785,0.719074,0.731088,0.748416
1,0.598016,0.606984,0.608505,0.597699
2,0.584914,0.621590,0.624869,0.628730
3,0.864519,0.806090,0.791853,0.782464


The different cutoffs give similar results:

    When the user does play popular songs a lot( pop ratio in the >75% quantile among all users), the churn rate
    become higher.
But from the table above, it seems the mean popular music play counts does not seem to easily distinguish churned users from loyal users. Now check the accumulated play_counts of popular music. 

In [128]:
popular_songs_cutoff = [100,500,1000,2000]
cutoff_quantile_dict = {}
for i in range(len(popular_songs_cutoff)):
    print 'procesing no.%d cutoff: %d' % (i,popular_songs_cutoff[i])
    num_pop_songs = sum(df_play.groupby('song_id_new').size()>popular_songs_cutoff[i])
    popular_songs = df_play['song_id_new'].value_counts()[:(num_pop_songs-1)].index
    df_play['is_popular'] = df_play.song_id_new.isin(set(popular_songs))
    df_temp = df_play[['uid', 'is_popular','label']].groupby('uid').\
              agg({'is_popular': ['sum'], 'label':['mean']}).\
              rename(columns = {'is_popular': 'is_popular_count', 'label': 'label'})
    df_temp.columns = df_temp.columns.get_level_values(0)
    df_temp.label = df_temp.label.apply(lambda x: x>0)
    cutoff_quantile_dict[i] = df_temp.groupby(pd.cut(df_temp.is_popular_count, \
                           np.percentile(df_temp.is_popular_count, [0,25,50,75,100]),include_lowest = True)).\
                           mean()['label']
cutoff_quantile_dict

procesing no.0 cutoff: 100
procesing no.1 cutoff: 500
procesing no.2 cutoff: 1000
procesing no.3 cutoff: 2000


{0: is_popular_count
 [0, 6]              0.864386
 (6, 20]             0.815432
 (20, 75]            0.677121
 (75, 30670]         0.403405
 Name: label, dtype: float64, 1: is_popular_count
 [0, 3]              0.849331
 (3, 14]             0.800635
 (14, 51]            0.695523
 (51, 25796]         0.416129
 Name: label, dtype: float64, 2: is_popular_count
 [0, 2]              0.834818
 (2, 11]             0.791746
 (11, 40]            0.710228
 (40, 23578]         0.424929
 Name: label, dtype: float64, 3: is_popular_count
 [0, 1]              0.824643
 (1, 8]              0.782513
 (8, 29]             0.711773
 (29, 10072]         0.439912
 Name: label, dtype: float64}

When using 2000 as the cutoff play counts for popular songs, the 25% and 75% quantiles of users' popular
song values, 8 and 29 can differentiate the users significantly. Say n=played_times:

    When n < 8, churn possibility is high.

    When n > 29, churn possibility is low.

When using 100 as the cutoff play counts,25%, 75% quantile cutoffs are 7 and 75 for defining popular songs as played more than 100 times. Say n=played_times:

    When n < 7, churn possibility is high.

    When n > 75, churn possibility is low.
Looks like 100 works the best as the last quantile gives the smallest churn-rate.

In [129]:
# cutoff value for popular song play count is 100
popular_song_cutoff = 100
popular_songs_num = sum(df_play.groupby('song_id_new').size() > popular_song_cutoff)
popular_songs = df_play['song_id_new'].value_counts()[:(popular_songs_num-1)].index
df_play['is_popular'] = df_play.song_id_new.isin(set(popular_songs))


In [130]:
df_temp = df_play[['uid','is_popular','label']].groupby('uid').sum()
df_temp.label = df_temp.label.apply(lambda x: int(x>0))
df_temp.groupby(pd.cut(df_temp.is_popular, \
                           np.percentile(df_temp.is_popular, [0, 25,75, 100]),include_lowest = True)).mean()['label']

is_popular
[0, 6]         0.864386
(6, 75]        0.743564
(75, 30670]    0.403405
Name: label, dtype: float64

In [131]:
df['most_popular_count'] = list(df_play.groupby('uid').sum()['is_popular'])
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count,total_play_count,least_popular_ratio,range,most_popular_count
0,100052111,0.107143,ar,0,0.107143,0.000000,1,0-100,0.000000,0,112,0.000,"[0, 0.018]",58
1,100107633,0.500000,ar,0,0.500000,3.033333,1,0-100,45.500000,0,4,0.000,"[0, 0.018]",4
2,100223381,0.000000,ar,0,0.000000,33.333333,1,0-100,125.000000,6,16,0.375,"(0.069, 1]",0
3,100581940,0.800000,ar,0,0.800000,17.200000,1,0-100,51.600000,1,20,0.050,"(0.018, 0.069]",17
4,100733657,0.434783,ar,0,0.434783,374.766667,1,100-400,195.530435,0,115,0.000,"[0, 0.018]",72


#### Examine the count of plays in different time window


In [132]:
df_daily_count = df_play.groupby(['uid','date']).size().unstack().fillna(0)

In [133]:
df_daily_count.head()

date,20170301,20170302,20170303,20170304,20170305,20170306,20170307,20170308,20170309,20170330,...,20170412,20170413,20170414,20170415,20170416,20170417,20170418,20170419,20170420,20170421
uid,,,,,,,,,,,,,,,,,,,,,
100052111,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,112,0,0
100107633,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100223381,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100581940,0,0,0,0,0,0,0,0,0,20,...,0,0,0,0,0,0,0,0,0,0
100733657,16,6,16,0,32,0,15,9,21,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
days = df_daily_count.shape[1]
days

33

#### Divide the 33 days into 5 time window, each window is approximately 1 week.
(0,9], (9,12], (12,19], (19,26], (26, 33] are the days windows from the cut-off date 2017/04/22.

In [135]:
day_windows = [days, 26, 19, 12, 9, 0]


In [136]:
for i in xrange(len(day_windows)-1): 
    for j in range(day_windows[i+1],day_windows[i]):
        print j,
    print
        

26 27 28 29 30 31 32
19 20 21 22 23 24 25
12 13 14 15 16 17 18
9 10 11
0 1 2 3 4 5 6 7 8


In [137]:
for i in xrange(len(day_windows)-1):
    if i == len(day_windows)-2:
        colname = 'count_in_recent_'+str(i+4)+'week'
    else:
        colname = 'count_in_recent_'+str(i+1)+'week'
    df[colname] = list(df_daily_count.iloc[:,day_windows[i+1]:day_windows[i]].sum(axis = 1).astype('int'))
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count,total_play_count,least_popular_ratio,range,most_popular_count,count_in_recent_1week,count_in_recent_2week,count_in_recent_3week,count_in_recent_4week,count_in_recent_8week
0,100052111,0.107143,ar,0,0.107143,0.000000,1,0-100,0.000000,0,112,0.000,"[0, 0.018]",58,112,0,0,0,0
1,100107633,0.500000,ar,0,0.500000,3.033333,1,0-100,45.500000,0,4,0.000,"[0, 0.018]",4,0,0,4,0,0
2,100223381,0.000000,ar,0,0.000000,33.333333,1,0-100,125.000000,6,16,0.375,"(0.069, 1]",0,0,1,15,0,0
3,100581940,0.800000,ar,0,0.800000,17.200000,1,0-100,51.600000,1,20,0.050,"(0.018, 0.069]",17,0,0,0,20,0
4,100733657,0.434783,ar,0,0.434783,374.766667,1,100-400,195.530435,0,115,0.000,"[0, 0.018]",72,0,0,0,0,115


In [138]:
df.shape

(150849, 19)

In [143]:
df = df.drop('range',1)

In [144]:
df.shape

(150849, 18)

In [145]:
df.tail()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count,total_play_count,least_popular_ratio,most_popular_count,count_in_recent_1week,count_in_recent_2week,count_in_recent_3week,count_in_recent_4week,count_in_recent_8week
150844,99633829,0.468468,ar,0,0.468468,1011.900000,1,1k-7k,182.324324,1,333,0.003003,276,0,0,0,0,333
150845,99689252,0.080745,ar,0,0.080745,72.366667,0,0-100,26.968944,21,161,0.130435,42,2,0,25,134,0
150846,99725077,0.076923,ar,0,0.076923,27.616667,0,0-100,63.730769,0,26,0.000000,9,0,0,14,12,0
150847,99983627,0.207547,ar,0,0.207547,298.833333,1,100-400,112.767296,1,159,0.006289,108,0,0,0,0,159
150848,99995513,0.000000,ar,0,0.000000,2.266667,1,0-100,136.000000,0,1,0.000000,0,0,0,0,1,0


In [146]:
labels = list(df['label'])
df = df.drop(['label','least_popular_ratio','total_play_count'],axis = 1)
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,play_time_label,avg_play_time,least_pop_count,most_popular_count,count_in_recent_1week,count_in_recent_2week,count_in_recent_3week,count_in_recent_4week,count_in_recent_8week
0,100052111,0.107143,ar,0,0.107143,0.000000,0-100,0.000000,0,58,112,0,0,0,0
1,100107633,0.500000,ar,0,0.500000,3.033333,0-100,45.500000,0,4,0,0,4,0,0
2,100223381,0.000000,ar,0,0.000000,33.333333,0-100,125.000000,6,0,0,1,15,0,0
3,100581940,0.800000,ar,0,0.800000,17.200000,0-100,51.600000,1,17,0,0,0,20,0
4,100733657,0.434783,ar,0,0.434783,374.766667,100-400,195.530435,0,72,0,0,0,0,115


In [147]:
df['label'] = labels
df.label = df.label.astype(int)
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,play_time_label,avg_play_time,least_pop_count,most_popular_count,count_in_recent_1week,count_in_recent_2week,count_in_recent_3week,count_in_recent_4week,count_in_recent_8week,label
0,100052111,0.107143,ar,0,0.107143,0.000000,0-100,0.000000,0,58,112,0,0,0,0,1
1,100107633,0.500000,ar,0,0.500000,3.033333,0-100,45.500000,0,4,0,0,4,0,0,1
2,100223381,0.000000,ar,0,0.000000,33.333333,0-100,125.000000,6,0,0,1,15,0,0,1
3,100581940,0.800000,ar,0,0.800000,17.200000,0-100,51.600000,1,17,0,0,0,20,0,1
4,100733657,0.434783,ar,0,0.434783,374.766667,100-400,195.530435,0,72,0,0,0,0,115,1


In [148]:
df.shape

(150849, 16)

#### Save dataframe as pkl

In [149]:
filename_pickle = '/Users/ZhijingYe/Desktop/data/output/play_full_features_and_label.pkl'
df.to_pickle(filename_pickle)

In [152]:
filename_pickle = '/Users/ZhijingYe/Desktop/data/output/play_log_features_and_label.pkl'
df.to_pickle(filename_pickle)

In [150]:
# test df is saved successfully
test_df = pd.read_pickle(filename_pickle)
test_df.shape

(150849, 16)

In [151]:
test_df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,play_time_label,avg_play_time,least_pop_count,most_popular_count,count_in_recent_1week,count_in_recent_2week,count_in_recent_3week,count_in_recent_4week,count_in_recent_8week,label
0,100052111,0.107143,ar,0,0.107143,0.000000,0-100,0.000000,0,58,112,0,0,0,0,1
1,100107633,0.500000,ar,0,0.500000,3.033333,0-100,45.500000,0,4,0,0,4,0,0,1
2,100223381,0.000000,ar,0,0.000000,33.333333,0-100,125.000000,6,0,0,1,15,0,0,1
3,100581940,0.800000,ar,0,0.800000,17.200000,0-100,51.600000,1,17,0,0,0,20,0,1
4,100733657,0.434783,ar,0,0.434783,374.766667,100-400,195.530435,0,72,0,0,0,0,115,1
